<a href="https://colab.research.google.com/github/takerumarui/codesandbox-test/blob/main/soracompicture.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SORACOMから取得したデータをStreamlitを用いてWEBアプリにグラフ化して表示する


## ライブラリのインストール

streamlit、およびアプリの動作の確認に使用する「ngrok」をインストールする。

In [ ]:
!pip install streamlit==1.20.0 --quiet
!pip install pyngrok==4.1.1 --quiet

streamlit、およびngrokをインポートする。

In [ ]:
import streamlit as st
from pyngrok import ngrok

## ページのUI

In [ ]:
# API_USERNAME='keyId-eewmuNeyaqkfmTCCqiRqaHfBUuOUjYUy'
# API_PASSWORD='secret-qFDv8Gko60PesgXuIYTLCTgr1QQ0WCaWgkYzW1BeJw5Kx2qVz6j0oVELQvgH1a0K'
# API_EMAIL='moriwaki@cee.ehime-u.ac.jp'
# API_DATA_PASSWORD='Moriwakilab00'
# API_POST='https://api.soracom.io/v1/auth'
# URL01="https://api.soracom.io/v1/data/Subscribers/440103213407109"
# URL02="https://api.soracom.io/v1/data/Subscribers/440103215409283"
# URL03="https://api.soracom.io/v1/data/Subscribers/440103213490553"
# URL04="https://api.soracom.io/v1/data/Subscribers/440103268104071"
# URL05="https://api.soracom.io/v1/data/Subscribers/440103268364242"
# URL06="https://api.soracom.io/v1/data/Subscribers/440103268105072"
# URL07="https://api.soracom.io/v1/data/Subscribers/440103268108878"
# URL08="https://api.soracom.io/v1/data/Subscribers/440103268125910"
# URL09="https://api.soracom.io/v1/data/Subscribers/440103215420030"
# URL10="https://api.soracom.io/v1/data/Subscribers/440103268361339"

In [ ]:
%%writefile app.py
# 以下を「app.py」に書き込み
import pandas as pd
import matplotlib.pyplot as plt
import json
import requests
import datetime
import streamlit as st

# APIの認証情報を環境変数から取得
api_username='keyId-eewmuNeyaqkfmTCCqiRqaHfBUuOUjYUy'
api_password='secret-qFDv8Gko60PesgXuIYTLCTgr1QQ0WCaWgkYzW1BeJw5Kx2qVz6j0oVELQvgH1a0K'
api_email='moriwaki@cee.ehime-u.ac.jp'
api_data_password='Moriwakilab00'
api_post='https://api.soracom.io/v1/auth'
url01="https://api.soracom.io/v1/data/Subscribers/440103213407109"
url02="https://api.soracom.io/v1/data/Subscribers/440103215409283"
url03="https://api.soracom.io/v1/data/Subscribers/440103213490553"
url04="https://api.soracom.io/v1/data/Subscribers/440103268104071"
url05="https://api.soracom.io/v1/data/Subscribers/440103268364242"
url06="https://api.soracom.io/v1/data/Subscribers/440103268105072"
url07="https://api.soracom.io/v1/data/Subscribers/440103268108878"
url08="https://api.soracom.io/v1/data/Subscribers/440103268125910"
url09="https://api.soracom.io/v1/data/Subscribers/440103215420030"
url10="https://api.soracom.io/v1/data/Subscribers/440103268361339"

# Streamlit app
st.title('堂野窪地区　傾斜センサ')

#apiキーとトークンを作成
auth = (api_username, api_password)
headers = {'Content-Type': 'application/json'}
data = {'email': api_email, 'password': api_data_password}

response = requests.post(api_post, auth=auth, headers=headers, data=json.dumps(data))
response.raise_for_status()
auth_response = response.json()

api_key = auth_response['apiKey']
api_token = auth_response['token']

# Allow users to select the time range
selected_week = st.selectbox('閲覧したい週を選んでください', ['今週', '先週', '2週間前'])

# Calculate the time range based on the selected option
current_time = datetime.datetime.now()

if selected_week == '今週':
    date_start = current_time - datetime.timedelta(days=current_time.weekday())
elif selected_week == '先週':
    date_start = current_time - datetime.timedelta(days=current_time.weekday() + 7)
elif selected_week == '2週間前':
    date_start = current_time - datetime.timedelta(days=current_time.weekday() + 14)

# Set the start and end date times
date_start = date_start.replace(hour=0, minute=0, second=0, microsecond=0)
date_end = date_start + datetime.timedelta(days=7)

# Convert to Unix timestamps
unix_timestamp_ms_start = int(date_start.timestamp() * 1000)
unix_timestamp_ms_end = int(date_end.timestamp() * 1000)

headers = {
    "Content-Type": "application/json",
    "X-Soracom-API-Key": api_key,
    "X-Soracom-Token": api_token
}

params = {
    "limit": 100000, #取得できる最大のデータ数
    'from': unix_timestamp_ms_start,
    'to': unix_timestamp_ms_end
}

# Define URLs
urls = [eval(f"url{i:02d}") for i in range(1, 11)]

# Select a URL using a dropdown
selected_index = st.selectbox('閲覧したい傾斜センサを選んでください', range(1, 11))
selected_url = urls[selected_index - 1]

# Fetch data for the selected URL
response = requests.get(selected_url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()

    # Create DataFrame
    inclination = []
    for i in range(len(data)):
        inclination.append(data[i]['content'])

    for i in range(len(inclination)):
        tmp = inclination[i].split(sep=',')
        inclination[i] = tmp

    df = pd.DataFrame(inclination, columns=['日付', '傾斜角X', '傾斜角Y', '傾斜角Z', '電圧'])

    # Convert columns to appropriate data types
    df['日付'] = pd.to_datetime(df['日付'], errors='coerce')
    df['傾斜角X'] = pd.to_numeric(df['傾斜角X'], errors='coerce')
    df['傾斜角Y'] = pd.to_numeric(df['傾斜角Y'], errors='coerce')
    df['傾斜角Z'] = pd.to_numeric(df['傾斜角Z'], errors='coerce')
    df['電圧'] = pd.to_numeric(df['電圧'], errors='coerce')

    # Display the DataFrame
    st.write(df)

    # Allow users to select the y-axis data
    selected_y_axes = ['傾斜角X', '傾斜角Y', '電圧']
    axis_labels = {'傾斜角X': 'Angle_X', '傾斜角Y': 'Angle_Y', '電圧': 'Voltage'}

    # Create subplots for each selected y-axis
    fig, axes = plt.subplots(len(selected_y_axes), 1, figsize=(10, 6 * len(selected_y_axes)))

    # Plot scatter plots for selected_y-axes
    for i, selected_y_axis in enumerate(selected_y_axes):
        axes[i].scatter(df['日付'], df[selected_y_axis], marker='o', color='b')
        axes[i].set_ylabel(axis_labels[selected_y_axis])  # Use custom axis label
        axes[i].set_xlabel('Time')  # Set x-axis label for each subplot
        axes[i].grid(True)

    # Adjust layout to prevent clipping of labels
    plt.tight_layout()

    # Show the Matplotlib figure in Streamlit
    st.pyplot(fig)

else:
    st.error(f"Failed to fetch data from {selected_url}. Status code: {response.status_code}")


Overwriting app.py


## Authtokenの設定
ngrokで接続するために必要な「Authtoken」を設定する
以下のコードの
**!ngrok authtoken**
のの箇所を自分のAuthtokenに置き換える。
Authtokenは、ngrokのサイトに登録すれば取得することができる。
https://ngrok.com/

In [ ]:
!ngrok authtoken 2OBOzqYxlLz8eZLpuiO1D4psYVY_5EHU3k58a87jzhgVLKgTM

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


## アプリの起動
streamlitのrunコマンドでアプリを起動する。

In [ ]:
!streamlit run app.py &>/dev/null&  # 「&>/dev/null&」により、出力を非表示にしてバックグランドジョブとして実行

ngrokのプロセスを終了した上で、新たにポートを指定して接続する。
接続の結果、urlを取得する。
ngrokの無料プランでは、同時に1つのプロセスしか動かせないので、エラーが発生した場合は「ランタイム」→「セッションの管理」で不要なGoogleColabのセッションを終了する。

In [ ]:
ngrok.kill()  # プロセスの修了
url = ngrok.connect(port="8501")  # 接続

INFO:pyngrok.process:Killing ngrok process: 71204
2024-01-31 08:05:35.831 Killing ngrok process: 71204
INFO:pyngrok.process:ngrok process starting: 71724
2024-01-31 08:05:35.852 ngrok process starting: 71724
INFO:pyngrok.process:t=2024-01-31T08:05:35+0000 lvl=info msg="no configuration paths supplied"

2024-01-31 08:05:35.896 t=2024-01-31T08:05:35+0000 lvl=info msg="no configuration paths supplied"

INFO:pyngrok.process:t=2024-01-31T08:05:35+0000 lvl=info msg="using configuration at default config path" path=/root/.ngrok2/ngrok.yml

2024-01-31 08:05:35.905 t=2024-01-31T08:05:35+0000 lvl=info msg="using configuration at default config path" path=/root/.ngrok2/ngrok.yml

INFO:pyngrok.process:t=2024-01-31T08:05:35+0000 lvl=info msg="open config file" path=/root/.ngrok2/ngrok.yml err=nil

2024-01-31 08:05:35.913 t=2024-01-31T08:05:35+0000 lvl=info msg="open config file" path=/root/.ngrok2/ngrok.yml err=nil

INFO:pyngrok.process:t=2024-01-31T08:05:35+0000 lvl=info msg="starting web service"

## 動作の確認
URLのhttpの部分をhttpsに変換する関数を設定


In [ ]:
def convert_http_to_https(url):
    if url.startswith("http://"):
        url = url.replace("http://", "https://", 1)
    return url

2024-01-31 08:05:36.205 t=2024-01-31T08:05:36+0000 lvl=info msg=end pg=/api/tunnels id=345cbb84f21d7b38 status=201 dur=91.307153ms



変換したurlを表示し、リンク先でアプリが動作することを確認する。

In [ ]:
print(convert_http_to_https(url))

https://afc3-35-196-156-206.ngrok-free.app
